In [9]:
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
    features = batch[b'data'].reshape((len(batch[b'data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch[b'labels']
    return features, labels

class SimpleCNN:
    def __init__(self, saved_model):
        self.conv1_filters = saved_model['conv1_filters']
        self.conv1_bias = saved_model['conv1_bias']
        self.fc1_weights = saved_model['fc1_weights']
        self.fc1_bias = saved_model['fc1_bias']
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=0, keepdims=True))
        return exp_x / np.sum(exp_x, axis=0, keepdims=True)
    
    def conv2d(self, x, filters, bias):
        n_filters, n_channels, f_height, f_width = filters.shape
        n_samples, height, width, _ = x.shape
        
        out_height = height - f_height + 1
        out_width = width - f_width + 1
        
        output = np.zeros((n_samples, out_height, out_width, n_filters))
        
        for i in range(out_height):
            for j in range(out_width):
                x_slice = x[:, i:i+f_height, j:j+f_width, :]
                for k in range(n_filters):
                    output[:, i, j, k] = np.sum(x_slice * filters[k].T, axis=(1,2,3)) + bias[k]
        
        return output
    
    def forward(self, x):
        conv1 = self.conv2d(x, self.conv1_filters, self.conv1_bias)
        relu1 = self.relu(conv1)
        flattened = relu1.reshape(x.shape[0], -1)
        fc1 = np.dot(flattened, self.fc1_weights) + self.fc1_bias.T
        output = self.softmax(fc1.T).T
        return output

# Load the saved model
model_file = 'simple_cnn_model.pkl'
with open(model_file, 'rb') as f:
    saved_model = pickle.load(f)

# Create model instance with loaded weights
model = SimpleCNN(saved_model)

# Load test data
data_dir = 'cifar-10-batches-py'  # Replace with your actual path
X_test, y_test = load_cifar10_batch(data_dir + '/test_batch')

# Preprocess test data
X_test = X_test.astype('float32') / 255.0

# Make predictions
y_pred_proba = model.forward(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Accuracy: 0.3655
Precision: 0.4520
Recall: 0.3655


/Users/sayeedkhan/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
